<a href="https://colab.research.google.com/github/MateusFerroAntunesdeOliveira/FeatureExtraction_MachineLearning/blob/main/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import pandas as pd
import re
import math
from skimage import feature
from skimage.feature import hog
from imutils import paths
from google.colab.patches import cv2_imshow
import os

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

model = InceptionV3(include_top = False, weights = 'imagenet', pooling = 'avg', input_tensor=Input(shape=(299,299,3)))
#model = Xception(include_top=False, weights='imagenet', pooling='avg')


# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= List of paths =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
file_list = []
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/humanos"))
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/praia"))
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/obras"))
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/onibus"))
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/dino"))
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/elefante"))
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/flores"))
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/cavalos"))
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/montanhas"))
# file_list.append(os.listdir(r"/content/drive/MyDrive/TDE_01/Base/comida"))

# general path
path = '/content/drive/MyDrive/TDE_01/Base/'

# list of classes
class_names = ['humanos', 'praia', 'obras', 'onibus', 'dino', 'elefante', 'flores', 'cavalos', 'montanhas', 'comida'] 


# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= Feature extraction =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
X = []
X_deep = []
y = []

for classes_files, classe in zip (file_list, range(10)):
    for i in range(100):
      name= str(path) + str(class_names[classe]) + str('/') + str(classes_files[i]) 
      print(name)
      imagem = cv2.imread(name)
      # imagem = cv2.resize(imagem,(299,299))  #### <=====

      print(imagem.shape)
      altura, largura, _ = imagem.shape
    
      # -------------- Convert the image to RGB and Gray -------------- 
      cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
      rgb   = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

      # -------------- Color Histograms -------------- 
      r_histograma = cv2.calcHist([rgb], [0], None, [256], [0, 256])/(altura*largura)
      g_histograma = cv2.calcHist([rgb], [1], None, [256], [0, 256])/(altura*largura)
      b_histograma = cv2.calcHist([rgb], [2], None, [256], [0, 256])/(altura*largura)

      # -------------- Local Binary Pattern (LBP) -------------- 
      lbp = feature.local_binary_pattern(cinza, 59, 1, method="uniform")
      (lbp_histograma, _) = np.histogram(lbp.ravel(), bins=59, range=(0, 59))
      lbp_histograma = lbp_histograma.astype("float")
      lbp_histograma /= (lbp_histograma.sum())
    
      # -------------- Hog (Histogram of Gradient - Direction) -------------- 
      hg = hog(cinza, orientations=8, pixels_per_cell=(32, 32), cells_per_block=(8, 8), block_norm='L2-Hys')
    
      # -------------- Concatenate the handcrafted feature sets -------------- 
      X_image = [lbp_histograma, hg, r_histograma, g_histograma, b_histograma]    
      X_image_aux = []
      for aux in X_image:
          X_image_aux = np.append(X_image_aux, np.ravel(aux))
    
      X_image = [i for i in X_image_aux]
      y.append(classe)
      X.append(X_image)
      
      # -------------- Extract deep features using InceptionV3 pretrained model -------------- 
      img = cv2.resize(imagem,(299,299))
      # img=imagem  ###### <<<=====
      xd = image.img_to_array(img)
      xd = np.expand_dims(xd, axis=0)
      xd = preprocess_input(xd)
      deep_features = model.predict(xd)
      print(deep_features.shape)
      
      X_image_aux = []
      for aux in deep_features:
          X_image_aux = np.append(X_image_aux, np.ravel(aux))
    
      deep_features = [i for i in X_image_aux]
      
      X_deep.append(deep_features)
            

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= Saving the files/folders =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Saving the extracted features (handcrafted) in a csv file
df = pd.DataFrame(X)
df.to_csv('X.csv', header=False, index=False)

# Saving the extracted features (deep) in a csv file
df = pd.DataFrame(X_deep)
df.to_csv('X_deep.csv', header = False, index = False)

# Saving the classes in a csv file
df_class = pd.DataFrame(y)
df_class.to_csv('y.csv', header = False, index = False)

In [ ]:
import pandas as pd
import numpy as np

# Labels
# y = pd.read_csv('/content/drive/MyDrive/TDE_01/Features_Cadu/y.csv', header=None)
# y = y.to_numpy()
# y = np.ravel(y)
# print(y.shape)

# handcrafted features
# X = pd.read_csv('/content/drive/MyDrive/TDE_01/Features_Cadu/X.csv', header=None)
# X = X.to_numpy()
# print(X.shape)

# deep features
# X = pd.read_csv('/content/drive/MyDrive/TDE_01/Features_Cadu/X_deep.csv', header=None)
# X = X.to_numpy()
# print(X.shape)


# ------------ x teste -------------
# X = pd.read_csv("/content/drive/MyDrive/TDE_01/s1.csv")
# X = X.to_numpy()
# print(X.shape)

# ------------ y teste -------------
# y = pd.read_csv('/content/drive/MyDrive/TDE_01/classes15', header = None)
# y = y.to_numpy()
# y = np.ravel(y)
# print(y.shape)


# ---------------------------------------------- PROVA ------------------------------------------------
# df = pd.read_csv (r'/content/drive/MyDrive/PROVA_01_ML/leaf.csv', delimiter = ',', header = None)
# dataset = df.to_numpy()
# X = dataset[:,2:16]
# y = dataset[:,0]
# print(X.shape)
# print(y.shape)


In [ ]:
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= FUNÇÃO PRA PLOTAR GRÁFICO BONITIN =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
import matplotlib.pyplot as plt
import matplotlib as pl
import seaborn as sns
import itertools
import matplotlib

matplotlib.rc('figure', figsize = (11, 11))

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
  plt.imshow(cm, interpolation = 'nearest',cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation = 45)
  plt.yticks(tick_marks, classes)
  
  if normalize:
    cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    print("Normalized confusion matrix")
  else:
    print("Confusion matrix, without normalization")

  thresh = cm.max()/2
  for i, j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(j, i, cm[i,j], horizontalalignment="center", color="white" if cm[i,j]>thresh else "black")
   
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')


  

**CLASSIFICADORES DE MACHINE LEARNING - USANDO HOLDOUT E VALIDAÇÃO CRUZADA**

In [ ]:
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-= CLASSIFIERS =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score
from sklearn import model_selection

#-------------------------------------------------
# EXAMPLE - How to Load DataSets
# from sklearn.datasets import load_diabetes

# X, y = load_diabetes(return_X_y = True)

# diabetes = load_diabetes()
# X, y = diabetes['data'], diabetes['target']
#-------------------------------------------------


# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= HOLDOUT =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

# -------------------- DecisionTree --------------------
# clfa = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10, min_samples_split = 10)
# clfa = clfa.fit(X_train, y_train)

# -------------------- KNN ----------------------------
# clfa = KNeighborsClassifier(n_neighbors = 1, p = 3, algorithm = 'brute', weights = 'distance')
# clfa = clfa.fit(X_train, y_train)

# -------------------- NaiveBayes --------------------
# clfa = GaussianNB(var_smoothing = 1e-9)
# clfa = clfa.fit(X_train, y_train)

# -------------------- SVM --------------------------- 
# parameters = [{'C': [0.1, 0.5, 1, 10, 100, 500, 1000], 
#                'kernel': ['linear']},
#               {'C': [0.1, 0.5, 1, 10, 100, 500, 1000], 
#                'gamma': [0.1, 0.001, 0.0001, 0.00001], 
#                'kernel': ['rbf']}]

# clfa = SVC(probability = True)
# clfa = SVC(probability = True, C = 0.1, kernel = 'linear')
# clfa = GridSearchCV(clfa, parameters, scoring = 'accuracy', cv = 5)
# clfa = clfa.fit(X_train, y_train)
# print(clfa.best_params_)

# -------------------- MLP ------------------------- 
# clfa = MLPClassifier(hidden_layer_sizes = (16), activation = 'relu', 
#                      max_iter = 5000, solver = 'lbfgs', tol =  1e-10, 
#                      early_stopping = True, validation_fraction = 0.2)
# clfa = clfa.fit(X_train, y_train)


# ---------------- PADRÃO PARA TODOS OS CLASSIFICADORES DE HOLDOUT -------------------
# predicted = clfa.predict(X_test)
# predp = clfa.predict_proba(X_test)
# score = clfa.score(X_test, y_test)
# matrix = confusion_matrix(y_test, predicted)

# print("=-"*20)
# print("Resultados baseados em HOLDOUT")
# print("Taxa de Acerto: %.5f" % score)

# print("\nMatriz de confusão:")
# print(matrix)
# print("\n\n")


# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-= VALIDAÇÃO CRUZADA =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

# -------------- DecisionTree --------------
# clfb = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10, min_samples_split = 10)
# folds = 5

# -------------- KNN -----------------------
# clfb = KNeighborsClassifier(n_neighbors = 100, p = 2, algorithm = 'kd_tree', weights = 'distance')
# folds = 5

# ------------ NaiveBayes ------------------
# clfb = GaussianNB(var_smoothing = 1e-12)
# folds = 5

# ------------ SVM ------------------------- 
# clfb = SVC(probability = True, C = 250, kernel = 'linear')
# clfb = SVC(probability = True)
# folds = 5
# clfb = GridSearchCV(clfb, parameters, scoring = 'accuracy', cv = folds)

# ------------ MLP ------------------------- 
# clfb = MLPClassifier(hidden_layer_sizes = (16), activation = 'relu', max_iter = 5000, 
#                      solver = 'sgd', tol = 1e-10, early_stopping = True, validation_fraction = 0.2)
# folds = 10

# ---------------- PADRÃO PARA TODOS OS CLASSIFICADORES DE VALIDAÇÃO CRUZADA -------------------
# result = model_selection.cross_val_score(clfb, X, y, cv = folds)
# Z = model_selection.cross_val_predict(clfb, X, y, cv = folds)
# f1Score = f1_score(y, Z, pos_label = 0, average = 'binary')
# cm = confusion_matrix(y, Z)

# print("=-"*20)
# print("Resultados baseados em Validacao Cruzada")
# print("Taxa de Acerto: %.5f" % result.mean())
# print("Desvio padrao: %.5f" % result.std())
# print("F1_score: %.5f" % f1Score)
# print("=-"*20)

# print("\nMatriz de confusão:")
# print(cm)




**REGRESSORES DE MACHINE LEARNING - USANDO HOLDOUT E VALIDAÇÃO CRUZADA**

In [ ]:
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-= REGRESSORS =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import model_selection

#-------------------------------------------------
# Exemple - How to Load DataSets
# from sklearn.datasets import load_diabetes

# X, y = load_diabetes(return_X_y = True)

# diabetes = load_diabetes()
# X, y = diabetes['data'], diabetes['target']
#-------------------------------------------------



# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= HOLDOUT =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

# --------------------- RegressionTree ---------------------
# regressor = DecisionTreeRegressor(criterion = 'mae', random_state = 1, max_depth = 3)
# regressor.fit(X_train, y_train)

# -------------- KNN_Regressor --------------
# regressor = KNeighborsRegressor(n_neighbors = 9)
# regressor.fit(X_train, y_train)

# -------------- SVR_Regressor --------------
# regressor = SVR(gamma = 'scale', kernel = 'rbf', C = 100)
# regressor.fit(X_train, y_train)

# -------------- MLP_Regressor --------------
# regressor = MLPRegressor(hidden_layer_sizes = {6}, random_state = 46, activation = 'relu', max_iter = 1000, solver = 'sgd')
# regressor.fit(X_train, y_train)


# ---------------- PADRÃO PARA TODOS OS REGRESSORES DE HOLDOUT -------------------
# output_prediction = regressor.predict(X_test)
# print("\nAvaliação utilizando Holdout com 30% para teste")
# print("R2 score: %.2f" % r2_score(y_test, output_prediction))
# print("Mean absolute error: %.2f" % mean_absolute_error(y_test, output_prediction))
# print("Mean squared error: %.2f" % mean_squared_error(y_test, output_prediction))



# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-= VALIDAÇÃO CRUZADA =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

# -------------- RegressionTree --------------
# regressor1 = DecisionTreeRegressor(criterion = 'mae', random_state = 0, max_depth = 2, min_samples_split = 2)
# folds = 5

# -------------- KNN_Regressor --------------
# regressor1 = KNeighborsRegressor(n_neighbors = 9)
# folds = 5

# -------------- SVR_Regressor --------------
# regressor1 = SVR(gamma = 'scale', kernel = 'rbf', C = 100)
# folds = 5

# -------------- MLP_Regressor --------------
# regressor1 = MLPRegressor(hidden_layer_sizes = {6}, random_state = 46, activation = 'relu', max_iter = 1000, solver = 'sgd')
# folds = 5

# ---------------- PADRÃO PARA TODOS OS REGRESSORES DE VALIDAÇÃO CRUZADA -------------------
# result = model_selection.cross_val_score(regressor1, X, y, cv = folds)
# output_prediction = model_selection.cross_val_predict(regressor1, X, y, cv = folds)
# print("\nAvaliação usando Validação Cruzada com %d folds" % folds)
# print("R2 scores = " + str(result))
# print("R2 score médio: %.2f" % (result.mean()))
# print("Mean absolute error (MEA): %.2f" % mean_absolute_error(y, output_prediction))
# print("Mean squared error  (MSE): %.2f" % mean_squared_error(y, output_prediction))




In [ ]:
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= Plot mistakes (images) =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
print(predicted.shape)
for i in range(len(predicted)):
    if (predicted[i] != y_test[i]):
        print(i)
        print("predicted[i]:", predicted[i], class_names[predicted[i]])
        print("y_test[i]:", y_test[i], class_names[y_test[i]])
        dist=1
        j=0
        while (j<len(X) and dist !=0): 
            dist = np.linalg.norm(X[j]-X_test[i])
            j+=1
        print("Label:", y[j-1], class_names[y[j-1]], "  /  Prediction: ", predicted[i], class_names[predicted[i]], predp[i][predicted[i]])
        name = "/content/drive/MyDrive/TDE_01/Base/"+ str(class_names[y[j-1]]) + "/" + str(j)+ ".jpg" 
        print(name)
        im=cv2.imread(name)
        cv2_imshow(im)
        print("=============================================================================")
  
